$$p=\frac{\theta RT}{V-b}-\frac{a}{V(V+b)}$$
其中
$$a(T)=0.42748\frac{\theta^2R^2T_{c}^2}{p_c}\alpha(T),b(T)=0.08664\frac{\theta RT_c}{p_c}(\beta(T))=1$$

$p=\frac{\theta RT}{V-0.08664\frac{\theta RT_c}{p_c}}-\frac{0.42748\frac{\theta^2R^2T_c^2}{p_c}\alpha(T)}{V(V+0.08664\frac{\theta RT_c}{p_c})}$

In [1]:
import numpy as np
from math import *
import matplotlib.pyplot as plt
import cmath

R = 8.314
def ALPHA(omega,T,T_c):
	"""
	alpha(T)函数，传入偏心因子，对比温度
	"""
	T_r = T/T_c
	return (1+(0.48+1.574*omega-0.176*omega*omega)*(1-sqrt(T_r)))**2
def SOLVE_THREE(P,Q)->list:
	OMEGA = (-1+(sqrt(3))*1j)/(2)
	l1,l2 = -Q/2+cmath.sqrt((Q/2)**2+(P/3)**3),-Q/2-cmath.sqrt((Q/2)**2+(P/3)**3)
	X1 = l1**(1/3)+l2**(1/3)
	X2 = OMEGA*l1**(1/3)+OMEGA*OMEGA*l2**(1/3)
	X3 = OMEGA*OMEGA*l1**(1/3)+OMEGA*l2**(1/3)
	X = [X1,X2,X3]
	solve_list = []
	for num in X:
		if num.imag == 0: # 实数
			solve_list.append(num.real)
	return solve_list
def getTHETA(p,V,T,T_c,p_c,omega): # θ应为正值
	alpha = ALPHA(omega,T,T_c) # α(T)
	# 取 ax^3+bx^2+cx+d=0
	# 计算a -> 三次方
	A = 0.42748*alpha*0.08664*(R**3*T_c**3)/(p_c**2)
	# 计算b -> 二次方
	B = -0.42748*alpha*(R**2*T_c**2*V)/(p_c)+0.08664**2*p*V*(R**2*T_c**2)/(p_c**2)+0.08664*R*T_c*R*T*V/p_c
	# 计算c -> 一次项
	C = R*T*V*V
	# 计算d -> 常数项
	D = -p*V**3
	# 求解方程 θ^3+pθ+q=0
	P = (3*A*C-B**2)/(3*A*A)
	Q = (27*A*A*D-9*A*B*C+2*B**3)/(27*A**3)
	# Theta = np.array(SOLVE_THREE(P=P,Q=Q))-B/(3*A)
	# return Theta
	Theta = np.roots([A,B,C,D])
	solve_list = []
	for num in Theta:
		if num.imag == 0: # 实数
			solve_list.append(num.real)
	return solve_list[0]

In [2]:
import pandas as pd
def read_excel(filepath,kind):
	excel_file = pd.ExcelFile(filepath)
	active_sheet = excel_file.sheet_names[0]
	df = pd.read_excel(filepath,sheet_name=active_sheet)
	headers = df.columns.tolist()
	if kind == 1: # 列
		columns_data = {}
		for header in headers:
			columns_data[header] = df[header].tolist()
		return columns_data
	elif kind == 0: # 行
		rows_data = []
		for index, row in df.iterrows():
			rows_data.append(row.tolist())
		columns_data = {}
		columns_data[headers[0]] = headers[1:]
		for lists in rows_data:
			columns_data[lists[0]] = lists[1:]
		return columns_data
TIES_OF_MATTER = read_excel('./PVT_data/High precise EOS polar data.xlsx',0)
# Tc K  pc bar(10^5 Pa) Vc cm^3  w
T_C = np.array(TIES_OF_MATTER['Tc(K)'])[0:14] # K
V_C = np.array(TIES_OF_MATTER['Vc(cm3/mol)'][0:14])/(10**6) # m^3
P_C = np.array(TIES_OF_MATTER['Pc(bar)'][0:14])*(10**5) # Pa
M = np.array(TIES_OF_MATTER['M'])[0:14] # g/mol
OMEGA = np.array(TIES_OF_MATTER['w'])[0:14]

In [3]:
ThetaAll = []
PAll = []
VAll = []
TAll = []
for i in range(14):
	PVT = read_excel(f'./PVT_data/PVT_data_{i+1}.xlsx',1)
	keys = list(PVT.keys())
	P,V,T = PVT[keys[0]],PVT[keys[2]],PVT[keys[1]] # 分别是Pa K m^3
	theta_temp,ptemp,vtemp,ttemp = [],[],[],[]
	tc,vc,pc,omega = T_C[i],V_C[i],P_C[i],OMEGA[i]
	for k in range(len(P)):
		p,v,t = P[k],V[k],T[k]
		ptemp.append(p)
		vtemp.append(v)
		ttemp.append(t)
		thetat = getTHETA(p,v,t,tc,pc,omega)
		theta_temp.append(thetat)
	PAll.append(ptemp)
	VAll.append(vtemp)
	TAll.append(ttemp)
	ThetaAll.append(theta_temp)
print(ThetaAll)

[[0.9881798914722982, 0.9991723960514572, 0.9996130122621429, 0.9997608504081192, 0.9998347388932003, 0.9998790320269565, 0.999908536932253, 0.9999295978861096, 0.9999453850724318, 0.9999576585513141, 0.999967473715169, 0.9999755018110528, 0.9999821901212557, 0.9999878481710536, 0.9999926969668054, 0.9999968985257618, 1.0000005743247005, 1.000003817232873, 1.0000066994645787, 1.00000927801874, 0.9882992534828728, 0.9991240326466395, 0.9995841876190397, 0.9997404214083556, 0.9998189325668163, 0.9998661461904504, 0.9998976620207533, 0.9999201915610253, 0.9999370981566051, 0.9999502530942793, 0.9999607803119529, 0.9999693955922431, 0.9999765764053695, 0.9999826534622537, 0.9999878630746071, 0.9999923785920621, 0.9999963300611304, 0.9999998169514948, 1.0000029166434463, 1.0000056902369916, 0.9882015902687824, 0.9990647115629524, 0.9995502122847715, 0.9997166559671434, 0.999800664546514, 0.9998513116461698, 0.9998851750707124, 0.9999094109435717, 0.9999276137144735, 0.9999417866586596, 0.99

In [4]:
import itertools
# 根据文件顺序拼接
Theta_DataSet,M_DataSet,OMEGA_DataSet,T_r_DataSet,p_r_DataSet,Zc_DataSet,Delta_THETA_DataSet = [],[],[],[],[],[],[]
LENGTH = len(ThetaAll[0])
print(LENGTH)
for nui in range(14):
	Theta_DataSet = Theta_DataSet + ThetaAll[nui]
	M_DataSet = M_DataSet+[M[nui]]*LENGTH
	OMEGA_DataSet = OMEGA_DataSet+[OMEGA[nui]]*LENGTH
	T_r_DataSet = T_r_DataSet+list(np.array(TAll[nui])/T_C[nui])
	p_r_DataSet = p_r_DataSet+list(np.array(PAll[nui])/P_C[nui])
	Zc_DataSet = Zc_DataSet+[P_C[nui]*V_C[nui]/(R*T_C[nui])]*LENGTH
THETA_IDEAL = np.array(Zc_DataSet)/(0.3333)
Delta_THETA_DataSet = np.array(Theta_DataSet)/THETA_IDEAL
Delta_THETA_DataSet = list(Delta_THETA_DataSet)
DataSet = {
	'θ':Theta_DataSet,
	'M':M_DataSet,
	'w':OMEGA_DataSet,
	'Tr':T_r_DataSet,
	'pr':p_r_DataSet,
	'Zc':Zc_DataSet,
	'θ/θideal':Delta_THETA_DataSet
}

400


In [5]:
dfc = pd.DataFrame(DataSet)

# 将 DataFrame 写入 Excel 文件
filepath = './HANDLE_data/ThetaData.xlsx'
dfc.to_excel(filepath, index=False)